# init
used to query DB for spectra and labels

In [1]:
import os
from pathlib import Path

from dotenv import find_dotenv, load_dotenv

from pptoolbox.connectors import BaseEFSConnector, PFDBConnector

from datetime import datetime

In [2]:
load_dotenv(find_dotenv())

PF_SQL_PASSWORD = os.environ.get("PLATFORM_SQL_PASSWORD", None)  # PP_SQL_PASSWORD
PF_KEY_PATH = os.environ.get("DS_SERVER_KEYPATH", None)  # PP_SERVER_KEYPATH
PF_EFS_URL = os.environ.get("PF_EFS_URL", None)  # PP_EFS_URL

print(PF_SQL_PASSWORD,PF_KEY_PATH,PF_EFS_URL)

F398FYMLKN7XVVXJ C:/Users/RyanSoh/.ssh/datascience-research-instance-2.pem http://54.254.245.207:5014


In [3]:
raw_data_path = Path ("../data/raw")
ouptput_folder = raw_data_path / "v2"
ouptput_folder.mkdir(parents=True, exist_ok=True)

In [4]:
company_id = 1089
product_type = 6636
today_date = datetime.now().strftime("%y%m%d")

print (today_date)

250922


# label

In [5]:
BASE_QUERY = f"""
SELECT lnpv.lot_id, lot.name as lot_name, 
	   property.name as property_name,
       lnpv.value as property_value,
       lot.company_id,
       lot.product_type_id,
       product_type.name as product_name
FROM lot_numerical_property_value lnpv
INNER JOIN lot on lot.id = lnpv.lot_id
INNER JOIN numerical_property on numerical_property.id = lnpv.numerical_property_id
INNER JOIN property on numerical_property.property_id = property.id
INNER JOIN product_type on product_type.id = lot.product_type_id
WHERE lot.company_id = {company_id}
AND lot.product_type_id = {product_type}

UNION ALL

SELECT lopv.lot_id, lot.name as lot_name, 
	   property.name as property_name,
       lopv.value as property_value,
       lot.company_id,
       lot.product_type_id,
       product_type.name as product_name
FROM lot_option_property_value lopv
INNER JOIN lot on lot.id = lopv.lot_id
INNER JOIN option_property on option_property.id = lopv.option_property_id
INNER JOIN property on option_property.property_id = property.id
INNER JOIN product_type on product_type.id = lot.product_type_id
WHERE lot.company_id = {company_id}
AND lot.product_type_id = {product_type}

ORDER BY lot_id
"""

In [6]:
db_conn = PFDBConnector()
info_df = db_conn.query(PF_KEY_PATH, PF_SQL_PASSWORD, BASE_QUERY).set_index("lot_id")
print("successful query")

label_filename = ouptput_folder / f"label_{today_date}.csv"
info_df.to_csv(label_filename)

successful query


# spectra

In [7]:
BASE_QUERY = f"""
SELECT
	sp.id AS specimen_id,
	l.id AS lot_id,
	l.name AS lot_name,
	sp.date_scanned,
	sp.analyzer_id AS analyser_id,
    l.company_id as company_id,
	p.id AS product_id,
    p.name AS product_name
FROM
	specimen sp
	INNER JOIN lot l ON l.id = sp.lot_id
	INNER JOIN product_type p on l.product_type_id = p.id
WHERE l.company_id = {company_id}
AND p.id = {product_type}
ORDER BY l.id;
"""

In [8]:
SPECTRA_COLS = [
        "raw_data",
        "dark_ref_data",
        "white_ref_data",
        "dark_ref_scan_time",
        "white_ref_scan_time",
    ]
db_conn = PFDBConnector()
info_df = db_conn.query(PF_KEY_PATH, PF_SQL_PASSWORD, BASE_QUERY)
print("successful query")
efs_conn = BaseEFSConnector(url=PF_EFS_URL)
print(len(info_df))

# response = input(f'Found {info_df.shape[0]} rows. Proceed? [y]/n ')
# if response.lower() == 'n':
#     print('stopping data pull')
#     raise SystemExit()

spectra_df = efs_conn.fetch_spectra(info_df.specimen_id.values)
joined_df = info_df.merge(
    spectra_df.loc[:, SPECTRA_COLS], left_on="specimen_id", right_index=True
)
joined_df.set_index("lot_id", inplace=True)
print("Successfully queried from DB")
print(joined_df.head(n=10))
print(joined_df.shape)

spectra_filename = ouptput_folder / f"spectra_{today_date}.csv"
joined_df.to_csv(spectra_filename)

successful query
346


Fetching from EFS: 100%|██████████| 7/7 [00:26<00:00,  3.79s/it]


Successfully queried from DB
        specimen_id            lot_name  date_scanned  analyser_id  \
lot_id                                                               
89593        528114  Pilot27Jan_Ethanol    1737944898           78   
89593        528115  Pilot27Jan_Ethanol    1737945087           78   
89595        528118       HC001_Ethanol    1737949820           78   
89595        528119       HC001_Ethanol    1737949921           78   
89598        528123       HC002_Ethanol    1737954323           78   
89598        528124       HC002_Ethanol    1737954546           78   
90053        530245       HC003_Ethanol    1738642772           78   
90053        530246       HC003_Ethanol    1738642869           78   
90064        530273       HC004_Ethanol    1738646071           78   
90064        530278       HC004_Ethanol    1738646169           78   

        company_id  product_id           product_name  \
lot_id                                                  
89593         10